# Chapter 3: Exercise 14

**Import packages**

In [1]:
# import numpy as np
# import pandas as pd
# import statsmodels.api as sm
# import matplotlib.pyplot as plt
# from statsmodels.stats.anova import anova_lm
# from ISLP.models import (ModelSpec as MS, summarize, poly)